<a href="https://colab.research.google.com/github/Ucchwas/Apnea-ECG/blob/main/Apnea_ECG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wfdb
!pip install tsfel
!pip install biosppy
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from biosppy.signals import ecg
from biosppy.signals import resp
import tsfel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler
from tensorflow import keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.7 MB/s eta 0:00:00


In [ ]:
record_names = ['a01r', 'a02r', 'a03r', 'a04r', 'b01r', 'c01r', 'c02r', 'c03r']
records = []
for name in record_names:
    record = wfdb.rdrecord(name, channels=[0, 1, 2, 3], pn_dir='apnea-ecg')
    records.append(record)

In [ ]:
# Print all data with column names for each record
for idx, record in enumerate(records):
    print(f"Record {idx + 1} - {record.record_name}")
    for key, value in record.__dict__.items():
        print(f"{key}: {value}")
    print("\n")

Record 1 - a01r
record_name: a01r
n_sig: 4
fs: 100
counter_freq: None
base_counter: None
sig_len: 2956796
base_time: None
base_date: None
comments: []
sig_name: ['Resp C', 'Resp A', 'Resp N', 'SpO2']
p_signal: [[-0.1015  -0.3446   0.23545 98.     ]
 [-0.1025  -0.3455   0.2364  98.     ]
 [-0.1035  -0.3464   0.23735 98.     ]
 ...
 [ 0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.     ]]
d_signal: None
e_p_signal: None
e_d_signal: None
file_name: ['a01r.dat', 'a01r.dat', 'a01r.dat', 'a01r.dat']
fmt: ['16', '16', '16', '16']
samps_per_frame: [1, 1, 1, 1]
skew: [None, None, None, None]
byte_offset: [None, None, None, None]
adc_gain: [20000.0, 20000.0, 20000.0, 1.0]
baseline: [0, 0, 0, 0]
units: ['mV', 'mV', 'mV', 'mV']
adc_res: [16, 16, 16, 16]
adc_zero: [0, 0, 0, 0]
init_value: [-2030, -6892, 4709, 98]
checksum: [-21303, -27013, -10435, -4510]
block_size: [0, 0, 0, 0]


Record 2 - a02r
record_name: a02r
n_sig: 4
fs: 100
counter_

In [ ]:
# Initialize lists to store signals
resp_signals_C = []
resp_signals_A = []
resp_signals_N = []
spo2_signals = []

# Extract signals from each record and store them
for record in records:
    resp_signals_C.append(record.p_signal[:, 0])
    resp_signals_A.append(record.p_signal[:, 1])
    resp_signals_N.append(record.p_signal[:, 2])
    spo2_signals.append(record.p_signal[:, 3])

In [ ]:
# Initialize an empty list to store the annotations
annotations_list = []

# Loop through each record name and read the corresponding annotations
for name in record_names:
    annotations = wfdb.rdann(name, extension='apn', pn_dir='apnea-ecg')
    annotations_list.append(annotations)

In [ ]:
for idx, annotations in enumerate(annotations_list):
    print(f"Annotations for Record {idx + 1} - {annotations.record_name}")
    #print(f"Sample Labels: {annotations.sample}")
    print(f"Symbol Labels: {annotations.symbol}\n")

Annotations for Record 1 - a01r
Symbol Labels: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A

In [ ]:
# Initialize variables to store the counts
total_apnea_count = 0
total_non_apnea_count = 0

# Loop through each set of annotations in the list
for annotations in annotations_list:
    # Extract the annotation labels
    annotation_labels = annotations.symbol
    # Count the occurrences of specific labels
    num_apnea = annotation_labels.count('A')
    num_non_apnea = annotation_labels.count('N')

    # Accumulate the counts
    total_apnea_count += num_apnea
    total_non_apnea_count += num_non_apnea

# Print the total counts
print(f"Total Number of Apnea signals: {total_apnea_count}")
print(f"Total Number of Non-apnea signals: {total_non_apnea_count}")

Total Number of Apnea signals: 1609
Total Number of Non-apnea signals: 2346


In [ ]:
# Initialize empty list to store features
two_features = []

# Loop through each record and corresponding annotations
for record, annotations in zip(records, annotations_list):
    # Extract signals
    resp_signals_C = record.p_signal[:, 3]

    # Extract features and labels
    features = []
    for i in range(len(annotations.sample)):
        start = annotations.sample[i]
        end = annotations.sample[i + 1] if i < len(annotations.sample) - 1 else len(resp_signals_C)
        segment = resp_signals_C[start:end]
        mean = np.mean(segment)
        std = np.std(segment)
        label = annotations.symbol[i]  # Assuming labels are in annotations.symbol
        features.append((mean, std, label))

    # Append features for the current record to the overall list
    two_features.extend(features)
len(two_features)

3955

In [ ]:
# import numpy as np

# # Initialize empty list to store all features
# extracted_features_list = []

# # Loop through each record and corresponding annotations
# for record, annotations in zip(records, annotations_list):
#     # Extract respiratory signals
#     resp_signals_C = record.p_signal[:, 0]

#     # Extract features and labels
#     for i in range(len(annotations.sample)):
#         start = annotations.sample[i]
#         end = annotations.sample[i + 1] if i < len(annotations.sample) - 1 else len(resp_signals_C)
#         segment = resp_signals_C[start:end]

#         # Time domain features
#         mean = np.mean(segment)
#         std = np.std(segment)
#         skewness = np.mean((segment - mean) ** 3) / (np.std(segment) ** 3)
#         area_abs = np.sum(np.abs(segment))

#         # Peak-related features
#         peak_heights = segment - np.min(segment)
#         mean_peak_height = np.mean(peak_heights)
#         std_peak_height = np.std(peak_heights)
#         skewness_peak_height = np.mean((peak_heights - mean_peak_height) ** 3) / (np.std(peak_heights) ** 3)
#         num_peaks = len(segment)
#         mean_inter_peak_distance = len(segment) / np.sum(np.diff(np.where(peak_heights > 0)[0]))
#         std_peak_distance = np.std(np.diff(np.where(peak_heights > 0)[0]))
#         skewness_inter_peak_distance = np.mean((np.diff(np.where(peak_heights > 0)[0]) - mean_inter_peak_distance) ** 3) / (np.std(np.diff(np.where(peak_heights > 0)[0])) ** 3)
#         sum_peak_heights = np.sum(peak_heights)
#         peak_frequency = len(segment) / np.sum(segment > (np.max(segment) / 2))

#         # Frequency domain features (example using FFT)
#         fft_result = np.fft.fft(segment)
#         magnitude_spectrum = np.abs(fft_result)
#         dominant_frequency = np.argmax(magnitude_spectrum)  # Dominant frequency component
#         power_spectrum = magnitude_spectrum ** 2  # Power spectrum
#         total_power = np.sum(power_spectrum)  # Total power
#         mean_frequency = np.sum(magnitude_spectrum * np.arange(len(magnitude_spectrum))) / np.sum(magnitude_spectrum)
#         central_frequency = np.argmax(magnitude_spectrum[len(magnitude_spectrum) // 2:]) + len(magnitude_spectrum) // 2

#         # Define low and high frequency indices for computing band power
#         # Adjust these values according to your requirements
#         low_frequency_index = 0
#         high_frequency_index = len(power_spectrum) // 2

#         band_power = np.sum(power_spectrum[low_frequency_index:high_frequency_index])  # Adjust low and high frequency index according to your requirements

#         # Store features along with label
#         label = 1 if annotations.symbol[i] == 'A' else 0  # Assuming labels are in annotations.symbol
#         features = [mean, std, skewness, area_abs, mean_peak_height, std_peak_height, skewness_peak_height,
#                     num_peaks, mean_inter_peak_distance, std_peak_distance, skewness_inter_peak_distance,
#                     sum_peak_heights, peak_frequency, mean_frequency, central_frequency, band_power, label]
#         extracted_features_list.append(features)

# # Print length of extracted features list
# print("Total number of extracted features:", len(extracted_features_list))

In [ ]:
!pip install pyentrp
import numpy as np
from scipy.stats import entropy
from scipy.signal import welch
from pyentrp import entropy as ent

# Initialize empty list to store all features
extracted_features_list = []

# Loop through each record and corresponding annotations
for record, annotations in zip(records, annotations_list):
    # Extract respiratory signals
    resp_signals_C = record.p_signal[:, 3]

    # Extract features and labels
    for i in range(len(annotations.sample)):
        start = annotations.sample[i]
        end = annotations.sample[i + 1] if i < len(annotations.sample) - 1 else len(resp_signals_C)
        segment = resp_signals_C[start:end]

        # Time domain features
        mean = np.mean(segment)
        std = np.std(segment)
        skewness = np.mean((segment - mean) ** 3) / (np.std(segment) ** 3)
        area_abs = np.sum(np.abs(segment))
        kurtosis = np.mean((segment - mean) ** 4) / (np.std(segment) ** 4)  # Kurtosis
        min_value = np.min(segment)  # Minimum value
        max_value = np.max(segment)  # Maximum value
        rms = np.sqrt(np.mean(segment ** 2))  # Root Mean Square
        zcr = (np.diff(np.sign(segment)) != 0).sum()  # Zero Crossing Rate
        signal_energy = np.sum(segment ** 2)
        entropy_signal = entropy(segment)  # Shannon entropy

        # Peak-related features
        peak_heights = segment - np.min(segment)
        mean_peak_height = np.mean(peak_heights)
        std_peak_height = np.std(peak_heights)
        skewness_peak_height = np.mean((peak_heights - mean_peak_height) ** 3) / (np.std(peak_heights) ** 3)
        num_peaks = len(segment)
        mean_inter_peak_distance = len(segment) / np.sum(np.diff(np.where(peak_heights > 0)[0]))
        std_peak_distance = np.std(np.diff(np.where(peak_heights > 0)[0]))
        skewness_inter_peak_distance = np.mean((np.diff(np.where(peak_heights > 0)[0]) - mean_inter_peak_distance) ** 3) / (np.std(np.diff(np.where(peak_heights > 0)[0])) ** 3)
        sum_peak_heights = np.sum(peak_heights)
        peak_frequency = len(segment) / np.sum(segment > (np.max(segment) / 2))
        peak_indices = np.where(peak_heights > 0)[0]
        max_peak_height = np.max(peak_heights) if len(peak_indices) > 0 else 0  # Maximum peak height
        min_peak_height = np.min(peak_heights) if len(peak_indices) > 0 else 0  # Minimum peak height

        # Frequency domain features (example using FFT)
        fft_result = np.fft.fft(segment)
        magnitude_spectrum = np.abs(fft_result)
        dominant_frequency = np.argmax(magnitude_spectrum)  # Dominant frequency component
        power_spectrum = magnitude_spectrum ** 2  # Power spectrum
        total_power = np.sum(power_spectrum)  # Total power
        mean_frequency = np.sum(magnitude_spectrum * np.arange(len(magnitude_spectrum))) / np.sum(magnitude_spectrum)
        central_frequency = np.argmax(magnitude_spectrum[len(magnitude_spectrum) // 2:]) + len(magnitude_spectrum) // 2
        # Define low and high frequency indices for computing band power
        # Adjust these values according to your requirements
        low_frequency_index = 0
        high_frequency_index = len(power_spectrum) // 2
        band_power = np.sum(power_spectrum[low_frequency_index:high_frequency_index])  # Adjust low and high frequency index according to your requirements
        f, pxx = welch(segment)  # Power spectral density estimation
        spectral_centroid = np.sum(f * pxx) / np.sum(pxx)  # Spectral Centroid
        spectral_spread = np.sqrt(np.sum(((f - spectral_centroid) ** 2) * pxx) / np.sum(pxx))  # Spectral Spread
        spectral_skewness = np.sum(((f - spectral_centroid) ** 3) * pxx) / (spectral_spread ** 3)  # Spectral Skewness
        spectral_kurtosis = np.sum(((f - spectral_centroid) ** 4) * pxx) / (spectral_spread ** 4)  # Spectral Kurtosis

        # Statistical Features
        median = np.median(segment)
        percentile_25 = np.percentile(segment, 25)
        percentile_75 = np.percentile(segment, 75)
        data_range = np.max(segment) - np.min(segment)

        # Store features along with label
        label = 1 if annotations.symbol[i] == 'A' else 0  # Assuming labels are in annotations.symbol
        features = [mean, std, skewness, area_abs, kurtosis, min_value, max_value,
            rms, zcr, signal_energy, entropy_signal,
            mean_peak_height, std_peak_height, skewness_peak_height,
            num_peaks, mean_inter_peak_distance, std_peak_distance, skewness_inter_peak_distance,
            sum_peak_heights, peak_frequency, max_peak_height, min_peak_height,
            dominant_frequency, central_frequency, band_power,
            spectral_centroid, spectral_spread, spectral_skewness, spectral_kurtosis,
            median, percentile_25, percentile_75, data_range,
            label]

        extracted_features_list.append(features)

# Print length of extracted features list
print("Total number of extracted features:", len(extracted_features_list))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


<ipython-input-10-a6d04b419c1d>:42: RuntimeWarning: divide by zero encountered in scalar divide
  skewness_inter_peak_distance = np.mean((np.diff(np.where(peak_heights > 0)[0]) - mean_inter_peak_distance) ** 3) / (np.std(np.diff(np.where(peak_heights > 0)[0])) ** 3)
<ipython-input-10-a6d04b419c1d>:24: RuntimeWarning: invalid value encountered in scalar divide
  skewness = np.mean((segment - mean) ** 3) / (np.std(segment) ** 3)
<ipython-input-10-a6d04b419c1d>:26: RuntimeWarning: invalid value encountered in scalar divide
  kurtosis = np.mean((segment - mean) ** 4) / (np.std(segment) ** 4)  # Kurtosis
<ipython-input-10-a6d04b419c1d>:38: RuntimeWarning: invalid value encountered in scalar divide
  skewness_peak_height = np.mean((peak_heights - mean_peak_height) ** 3) / (np.std(peak_heights) ** 3)
<ipython-input-10-a6d04b419c1d>:40: RuntimeWarning: divide by zero encountered in scalar divide
  mean_inter_peak_distance = len(segment) / np.sum(np.diff(np.where(peak_heights > 0)[0]))
/usr/loc

Total number of extracted features: 3955


In [ ]:
# cfg = tsfel.get_features_by_domain()
# # Initialize an empty list to store features and labels
# extracted_features_list = []

# # Loop through each record and corresponding annotations
# for record, annotations in zip(records, annotations_list):
#     # Extract signals
#     resp_signals_A = record.p_signal[:, 1]
#     print("Record Name ", record.record_name)
#     # Extract features and labels
#     features = []
#     for i in range(len(annotations.sample)):
#         start = annotations.sample[i]
#         end = annotations.sample[i + 1] if i < len(annotations.sample) - 1 else len(resp_signals_A)
#         segment = resp_signals_A[start:end]
#         try:
#           segment_features = tsfel.time_series_features_extractor(cfg, segment, fs=100)
#           features_dict = segment_features.to_dict('records')[0]  # Convert the first row to a dict
#         except Exception as e:
#           print(f"Feature extraction failed for segment {i}: {e}")
#           features_dict = {}

#         label = 1 if annotations.symbol[i] == 'A' else 0
#         features_dict['label'] = label
#         features.append(features_dict)

#     # Append features for the current record to the overall list
#     extracted_features_list.extend(features)

In [ ]:
import pandas as pd

# Convert the list of extracted features into a DataFrame
df = pd.DataFrame(extracted_features_list)

# Fill NaN values with 0
df = df.fillna(0)

# Assign column names including the label column
column_names = ['Mean', 'Std', 'Skewness', 'Area_Abs', 'Kurtosis', 'Min_Value', 'Max_Value',
                'RMS', 'ZCR', 'Signal_Energy', 'Entropy_Signal',
                'Mean_Peak_Height', 'Std_Peak_Height', 'Skewness_Peak_Height',
                'Num_Peaks', 'Mean_Inter_Peak_Distance', 'Std_Peak_Distance', 'Skewness_Inter_Peak_Distance',
                'Sum_Peak_Heights', 'Peak_Frequency', 'Max_Peak_Height', 'Min_Peak_Height',
                'Dominant_Frequency', 'Central_Frequency', 'Band_Power',
                'Spectral_Centroid', 'Spectral_Spread', 'Spectral_Skewness', 'Spectral_Kurtosis',
                'Median', 'Percentile_25', 'Percentile_75', 'Data_Range',
                'Label']

# Rename the columns of the DataFrame
df.columns = column_names

# Save the DataFrame to a CSV file with labeled columns and excluding the index
df.to_csv('Apnea_ECG_Signals_Dataset.csv', index=False)

# Print the shape of the DataFrame
print("Shape of the DataFrame:", df.shape)

Shape of the DataFrame: (3955, 34)


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif

# Convert the extracted features list into a DataFrame
df_features = pd.DataFrame(extracted_features_list)

# Fill NaN values with 0
df_features = df_features.fillna(0)
# Replace infinite values with NaN
df_features = df_features.replace([np.inf, -np.inf], np.nan)
# Fill NaN values (previously infinite values) with 0
df_features = df_features.fillna(0)

# Separate features and labels
X = df_features.drop(columns=[33])
y = df_features[33]

# Perform feature selection using SelectKBest with ANOVA F-value as the scoring function
k = 33  # Number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(X, y)

# Get indices of selected features
selected_feature_indices = selector.get_support(indices=True)

# Get the names of selected features
selected_feature_names = X.columns[selected_feature_indices]

# Create a new DataFrame with selected features and labels
df_selected_features = df_features[selected_feature_names]
df_selected_features['label'] = y

# Save the DataFrame to a CSV file
df_selected_features.to_csv('Top_Features.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [21 22] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the DataFrame containing the selected features and labels
df_selected_features = pd.read_csv('Top_Features.csv')

# Separate features and labels
X = df_selected_features.drop(columns=['label'])
y = df_selected_features['label']

# # Define indices for splitting
# total_samples = len(df_selected_features)
# split_index = int(total_samples * 0.7)  # 70% for training, 30% for testing

# # Split the dataset based on indices
# X_train = X.iloc[:split_index]
# y_train = y.iloc[:split_index]
# X_test = X.iloc[split_index:]
# y_test = y.iloc[split_index:]

# Split the dataset randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Create DataFrames for training and testing sets
df_train = pd.concat([X_train_scaled_df, y_train.reset_index(drop=True)], axis=1)
df_test = pd.concat([X_test_scaled_df, y_test.reset_index(drop=True)], axis=1)

# Save the training and testing DataFrames to CSV files
df_train.to_csv('features_train.csv', index=False)
df_test.to_csv('features_test.csv', index=False)

In [ ]:
from tensorflow import keras
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
      keras.layers.Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
      keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
      keras.layers.Dense(2, activation='sigmoid')
    ])

    # Compile the model with a lower learning rate
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, class_weight={0: 1, 1: 1}, callbacks=[early_stopping])

    # Evaluate the model on the test set
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")

    # Make predictions on the test set
    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_test_original = np.argmax(y_test, axis=1)

    # Print classification report and confusion matrix
    print("Classification Report:")
    print(classification_report(y_test_original, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test_original, y_pred))

    return model, y_pred

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
# Load the dataset
df = pd.read_csv('Top_Features.csv')

# Extract features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode labels ('N' and 'A') to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Resample the dataset to handle imbalance
oversampler = RandomOverSampler(sampling_strategy='minority')
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size= 0.2, train_size=0.8, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
model, y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 3s 12ms/step - loss: 1.5773 - accuracy: 0.9037 - val_loss: 0.9467 - val_accuracy: 0.9507
Epoch 2/100
94/94 [==============================] - 0s 5ms/step - loss: 0.7071 - accuracy: 0.9490 - val_loss: 0.5238 - val_accuracy: 0.9441
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4312 - accuracy: 0.9500 - val_loss: 0.3694 - val_accuracy: 0.9454
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.3301 - accuracy: 0.9520 - val_loss: 0.3039 - val_accuracy: 0.9467
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.2874 - accuracy: 0.9484 - val_loss: 0.2705 - val_accuracy: 0.9441
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.2546 - accuracy: 0.9504 - val_loss: 0.2507 - val_accuracy: 0.9427
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.2421 - accuracy: 0.9480 - val_loss: 0.2436 - val_accuracy: 0.9454
Epoch 8/100


In [ ]:
model.save("Apnea.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("Apnea.h5")

In [ ]:
# convert to tensorflow lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("Apnea.tflite", "wb").write(tflite_model)

61480